## 日本語BERTでlivedoorニュースを教師あり学習で分類

In [1]:
# 乱数シードの固定

import os
import random
import numpy as np
import torch

SEED_VALUE = 1234  # これはなんでも良い
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)  # PyTorchを使う場合


### GPUの使用可能を確認

画面上部のメニュー ランタイム > ランタイムのタイプを変更 で、 ノートブックの設定 を開く

ハードウェアアクセラレータに GPU を選択し、 保存 する

In [2]:
# GPUの使用確認：True or False
torch.cuda.is_available()

# TrueならGPU使用可能

True

## 準備1：Livedoorニュースをダウンロードしてtsvファイル化

参考：https://github.com/yoheikikuta/bert-japanese/blob/master/notebook/finetune-to-livedoor-corpus.ipynb


In [3]:
# Livedoorニュースのファイルをダウンロード
!wget -nc "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"

File 'ldcc-20140209.tar.gz' already there; not retrieving.



In [4]:
# ファイルを解凍し、カテゴリー数と内容を確認
import tarfile
import os

# 解凍
tar = tarfile.open("ldcc-20140209.tar.gz", "r:gz")
tar.extractall("./data/livedoor/")
tar.close()

# フォルダのファイルとディレクトリを確認
files_folders = [name for name in os.listdir("./data/livedoor/text/")]
print(files_folders)

# カテゴリーのフォルダのみを抽出
categories = [name for name in os.listdir(
    "./data/livedoor/text/") if os.path.isdir("./data/livedoor/text/"+name)]

print("カテゴリー数:", len(categories))
print(categories)


['sports-watch', 'README.txt', 'peachy', 'movie-enter', 'dokujo-tsushin', 'CHANGES.txt', 'livedoor-homme', 'topic-news', 'it-life-hack', 'kaden-channel', 'smax']
カテゴリー数: 9
['sports-watch', 'peachy', 'movie-enter', 'dokujo-tsushin', 'livedoor-homme', 'topic-news', 'it-life-hack', 'kaden-channel', 'smax']


In [5]:
# ファイルの中身を確認してみる
file_name = "./data/livedoor/text/movie-enter/movie-enter-6255260.txt"

with open(file_name, encoding='utf-8') as text_file:
    text = text_file.readlines()
    print("0：", text[0])  # URL情報
    print("1：", text[1])  # タイムスタンプ
    print("2：", text[2])  # タイトル
    print("3：", text[3])  # 本文

    # 今回は4要素目には本文は伸びていないが、4要素目以降に本文がある場合もある


0： http://news.livedoor.com/article/detail/6255260/

1： 2012-02-07T09:00:00+0900

2： 新しいヴァンパイアが誕生！　ジョニデ主演『ダーク・シャドウ』の公開日が決定

3： 　こんなヴァンパイアは見たことがない！　ジョニー・デップとティム・バートン監督がタッグを組んだ映画『ダーク・シャドウズ（原題）』の邦題が『ダーク・シャドウ』に決定。日本公開日が5月19日に決まった。さらに、ジョニー・デップ演じるヴァンパイアの写真が公開された。



In [6]:
# 本文を取得する前処理関数を定義


def extract_main_txt(file_name):
    with open(file_name, encoding='utf-8') as text_file:
        # 今回はタイトル行は外したいので、3要素目以降の本文のみ使用
        text = text_file.readlines()[3:]

        # 3要素目以降にも本文が入っている場合があるので、リストにして、後で結合させる
        text = [sentence.strip() for sentence in text]  # 空白文字(スペースやタブ、改行)の削除
        text = list(filter(lambda line: line != '', text))
        text = ''.join(text)
        text = text.translate(str.maketrans(
            {'\n': '', '\t': '', '\r': '', '\u3000': ''}))  # 改行やタブ、全角スペースを消す
        return text


In [7]:
# リストに前処理した本文と、カテゴリーのラベルを追加していく
import glob

list_text = []
list_label = []

for cat in categories:
    text_files = glob.glob(os.path.join("./data/livedoor/text", cat, "*.txt"))

    # 前処理extract_main_txtを実施して本文を取得
    body = [extract_main_txt(text_file) for text_file in text_files]

    label = [cat] * len(body)  # bodyの数文だけカテゴリー名のラベルのリストを作成

    list_text.extend(body)  # appendが要素を追加するのに対して、extendはリストごと追加する
    list_label.extend(label)


In [8]:
# 0番目の文章とラベルを確認
print(list_text[0])
print(list_label[0])


ロンドン五輪・柔道男子60kg級で銀メダルを獲得した平岡拓晃が、テレビ朝日の五輪特別番組に出演した。「メダルは金ではなかったんですが、4年間頑張ってきた色なのかなと思います」と切り出した平岡。準々決勝では、フランスのミルを相手に大接戦を繰り広げたが、「ポイントを取られていたので、前に出るしかないと思ってですね、抱きついてでもポイントを取ろうと思った。オリンピック前の練習でポイントを取られた時の練習も少しやっていたので、それが出たと思います」と振り返った。また、「冷静だった？」という問いには、「慌ててもいたんですが、やることはひとつだなと腹は決まっていた」と語る平岡は、一本勝ちを決めた準決勝についても、「ここ（準々決勝）で一回自分は死んだんだと。負けてる試合だったので、なんとか勝ちを拾ってそのときに、僕は死んだんだと思って、準決勝も決勝も開き直った」と話す。さらに、ガルスチャンに敗れた決勝は「掬い投げにいこうとしたんですけど、掬っていく方向を間違えてしまった。もう少し真上に引き上げてから掬えばよかったんですけど、はさまれてしまって。自分は前に出ているわけだから勢いで回ってしまって。判断ミスでした」と反省の弁を語った。そして、母や妻への感謝を口にした平岡は「自分の中では金メダル獲りたかったです」と改めて話しながらも、「とりあえず男子柔道は全員が金メダルを獲ることを目標にやってきて、僕が流れを作るべきだったんですけど、それができなくて。すぐ選手村に帰って、みんなに申し訳ないって頭を下げて。明日からは選手のサポートに回ることだけを考えています」と続けた。
sports-watch


In [9]:
# pandasのDataFrameにする
import pandas as pd

df = pd.DataFrame({'text': list_text, 'label': list_label})

# 大きさを確認しておく（7,376文章が存在）
print(df.shape)

df.head()


(7376, 2)


,text,label
0,ロンドン五輪・柔道男子60kg級で銀メダルを獲得した平岡拓晃が、テレビ朝日の五輪特別番組に出...,sports-watch
1,4日放送、TBS「S1」では、「Weeklyストーブ 猛虎がMLBを語る」と題し、阪神タイガ...,sports-watch
2,17日に行なわれたセリエA第24節で、日本代表DF長友佑都が所属するインテルはホームでボロー...,sports-watch
3,5日、サッカー日本代表のアルベルト・ザッケローニ監督が宮市亮（ボルトン）を絶賛。「どんな性格...,sports-watch
4,2日、大手スポーツ紙は、なでしこリーグ・INAC神戸による「異例のお願い」として、優勝が懸か...,sports-watch


In [10]:
# カテゴリーの辞書を作成
dic_id2cat = dict(zip(list(range(len(categories))), categories))
dic_cat2id = dict(zip(categories, list(range(len(categories)))))

print(dic_id2cat)
print(dic_cat2id)

# DataFrameにカテゴリーindexの列を作成
df["label_index"] = df["label"].map(dic_cat2id)
df.head()

# label列を消去し、text, indexの順番にする
df = df.loc[:, ["text", "label_index"]]
df.head()


{0: 'sports-watch', 1: 'peachy', 2: 'movie-enter', 3: 'dokujo-tsushin', 4: 'livedoor-homme', 5: 'topic-news', 6: 'it-life-hack', 7: 'kaden-channel', 8: 'smax'}
{'sports-watch': 0, 'peachy': 1, 'movie-enter': 2, 'dokujo-tsushin': 3, 'livedoor-homme': 4, 'topic-news': 5, 'it-life-hack': 6, 'kaden-channel': 7, 'smax': 8}


,text,label_index
0,ロンドン五輪・柔道男子60kg級で銀メダルを獲得した平岡拓晃が、テレビ朝日の五輪特別番組に出...,0
1,4日放送、TBS「S1」では、「Weeklyストーブ 猛虎がMLBを語る」と題し、阪神タイガ...,0
2,17日に行なわれたセリエA第24節で、日本代表DF長友佑都が所属するインテルはホームでボロー...,0
3,5日、サッカー日本代表のアルベルト・ザッケローニ監督が宮市亮（ボルトン）を絶賛。「どんな性格...,0
4,2日、大手スポーツ紙は、なでしこリーグ・INAC神戸による「異例のお願い」として、優勝が懸か...,0


In [11]:
# 順番をシャッフルする
df = df.sample(frac=1, random_state=123).reset_index(drop=True)
df.head()

,text,label_index
0,9日、ソウルの日本大使館正門に62歳の韓国人男性が、1トントラックで突っ込むという事件が起き...,5
1,最近、新時代のモバイルノートと言われているUltrabook（ウルトラブック）の魅力的な新製...,6
2,東日本大震災以降、節電が求められている。この夏、皆さんもさまざまな工夫をしていることでしょう...,3
3,先日「リーダー」機能は意外に便利！ iOSのSafariを上手に使う方法【デジ通】で、Saf...,6
4,25日、大森南朋の口癖が「ナオ的には」であるという演劇関係者の談話をWeb版「女性自身」が報...,5


In [12]:
# tsvファイルで保存する

# 全体の2割の文章数
len_0_2 = len(df) // 5

# 前から2割をテストデータとする
df[:len_0_2].to_csv("./test.tsv", sep='\t', index=False, header=None)
print(df[:len_0_2].shape)

# 前2割からを訓練&検証データとする
df[len_0_2:].to_csv("./train_eval.tsv", sep='\t', index=False, header=None)
print(df[len_0_2:].shape)


(1475, 2)
(5901, 2)


In [13]:
# tsvファイルをダウンロードしたい場合
# from google.colab import files

# ダウンロードする場合はコメントを外す
# 少し時間がかかる（4MB）
# files.download("./test.tsv")


# ダウンロードする場合はコメントを外す
# 少し時間がかかる（18MB）
# files.download("./train_eval.tsv")


## 準備2：LivedoorニュースをBERT用のDataLoaderにする

Hugginfaceのリポジトリの案内とは異なり、torchtextを使用した手法で実装

In [14]:
# MeCabとtransformersの用意
# !apt install aptitude swig
# !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
# !pip install mecab-python3

# !pip install transformers==4.5.1

Defaulting to user installation because normal site-packages is not writeable


In [15]:
# !pip install torchtext

Defaulting to user installation because normal site-packages is not writeable


In [16]:
!pip list | grep torch
!pip list | grep transformer

torch               1.8.1
torchaudio          0.8.1
torchtext           0.9.1
torchvision         0.9.1
transformers        4.5.1


In [17]:
import torch
import torchtext  # torchtextを使用

#from transformers.tokenization_bert_japanese import BertJapaneseTokenizer
from transformers import BertJapaneseTokenizer
# 日本語BERTの分かち書き用tokenizerです
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')


In [18]:
# データを読み込んだときに、読み込んだ内容に対して行う処理を定義します

max_length = 512  # 東北大学_日本語版の最大の単語数（サブワード数）は512


def tokenizer_512(input_text):
    """torchtextのtokenizerとして扱えるように、512単語のpytorchでのencodeを定義。ここで[0]を指定し忘れないように"""
    return tokenizer.encode(input_text, max_length=512, return_tensors='pt', truncation=True)[0]


TEXT = torchtext.legacy.data.Field(sequential=True, tokenize=tokenizer_512, use_vocab=False, lower=False,
                            include_lengths=True, batch_first=True, fix_length=max_length, pad_token=0)
# 注意：tokenize=tokenizer.encodeと、.encodeをつけます。padding[PAD]のindexが0なので、0を指定します。

LABEL = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

# (注釈)：各引数を再確認
# sequential: データの長さが可変か？文章は長さがいろいろなのでTrue.ラベルはFalse
# tokenize: 文章を読み込んだときに、前処理や単語分割をするための関数を定義
# use_vocab：単語をボキャブラリーに追加するかどうか
# lower：アルファベットがあったときに小文字に変換するかどうか
# include_length: 文章の単語数のデータを保持するか
# batch_first：ミニバッチの次元を用意するかどうか
# fix_length：全部の文章をfix_lengthと同じ長さになるように、paddingします
# init_token, eos_token, pad_token, unk_token：文頭、文末、padding、未知語に対して、どんな単語を与えるかを指定


In [19]:
# 各tsvファイルを読み込み、分かち書きをしてdatasetにします
# 少し時間がかかります
# train_eval：5901個、test：1475個
dataset_train_eval, dataset_test = torchtext.legacy.data.TabularDataset.splits(
    path='.', train='train_eval.tsv', test='test.tsv', format='tsv', fields=[('Text', TEXT), ('Label', LABEL)])



In [20]:
# torchtext.data.Datasetのsplit関数で訓練データと検証データを分ける
# train_eval：5901個、test：1475個

dataset_train, dataset_eval = dataset_train_eval.split(
    split_ratio=1.0 - 1475/5901, random_state=random.seed(1234))

# datasetの長さを確認してみる
print(dataset_train.__len__())
print(dataset_eval.__len__())
print(dataset_test.__len__())


4426
1475
1475


In [21]:
# datasetの中身を確認してみる
item = next(iter(dataset_train))
print(item.Text)
print("長さ：", len(item.Text))  # 長さを確認 [CLS]から始まり[SEP]で終わる。512より長いと後ろが切れる
print("ラベル：", item.Label)


tensor([    2,    52,    53,  1141,     5, 10706,    11,  7123,    36,  5325,
        28738, 28821, 27603, 28583, 13074, 28583,   416,    61,    17,  3081,
        15031, 23049,   908,    38,     8,  1732,    95,    14,    41,    32,
            7,   343,    20,    10,     8,  1732,  1996,   104,   197,    53,
           14, 11802,     7,  5797,    11,  3723,    15,    16,  4481,     6,
         8141,  3900,  8072, 28865, 12402, 28865,     5, 23851, 25071,    14,
          176,   759,    11,  2372,    10,     8,  7168,     5,   622,    12,
         2906,    26,    20,    16,    21,    10,     5,     9,     6,  8141,
         7111,     5, 22825,  4187,     8,   306,     9,   654,    40,     5,
         2523,    12,     9,    17,   247,    11,   906,    15,    16,    21,
           10,    82,   759,    14,  3252,    26,    20,    16,    21,    10,
            8,  6978,  2670,     7,     9,   197,    53,     5,   859,    48,
           53,    14,  5294,     6,  3571,   813,    14, 28090, 

In [22]:
# datasetの中身を文章に戻し、確認

print(tokenizer.convert_ids_to_tokens(item.Text.tolist()))  # 文章
dic_id2cat[int(item.Label)]  # id


['[CLS]', '一', '人', '芸', 'の', '日本一', 'を', '決める', '「', '東洋', '##水', '##産', 'PRE', '##S', '##ENT', '##S', 'R', '-', '1', 'ぐ', '##らん', '##ぷり', '2012', '」', '。', '決勝', '戦', 'が', '20', '日', 'に', '行わ', 'れ', 'た', '。', '決勝', '進出', '者', '12', '人', 'が', '順番', 'に', 'ネタ', 'を', '披露', 'し', 'て', 'いき', '、', 'お笑い', 'コンビ', 'CO', '##W', '##CO', '##W', 'の', '多田', '健二', 'が', '初', '優勝', 'を', '決め', 'た', '。', '今回', 'の', '大会', 'で', '注目', 'さ', 'れ', 'て', 'い', 'た', 'の', 'は', '、', 'お笑い', '芸人', 'の', 'スギ', 'ちゃん', '。', '彼', 'は', '一般', 'から', 'の', '投票', 'で', 'は', '1', '位', 'を', '獲得', 'し', 'て', 'い', 'た', 'ため', '優勝', 'が', '期待', 'さ', 'れ', 'て', 'い', 'た', '。', 'ファイナル', 'ステージ', 'に', 'は', '12', '人', 'の', 'うち', '3', '人', 'が', '進み', '、', '審査', '委員', 'が', 'おもしろ', '##かっ', 'た', 'と', '思う', '芸人', 'を', '実際', 'に', '指名', 'する', 'という', '方法', 'で', '優勝', 'が', '決まる', '。', '多田', 'と', 'スギ', 'ちゃん', 'が', '引き分け', 'た', 'ため', '、', '採点', 'し', '直し', 'と', 'なり', '審査', '員', 'の', 'うち', 'の', '一', '人', 'で', 'ある', '木村', '祐', '##一', 'の', 'ポイント', 'が', '優勝', 

'kaden-channel'

In [23]:
# DataLoaderを作成します（torchtextの文脈では単純にiteraterと呼ばれています）
batch_size = 16  # BERTでは16、32あたりを使用する

dl_train = torchtext.legacy.data.Iterator(
    dataset_train, batch_size=batch_size, train=True)

dl_eval = torchtext.legacy.data.Iterator(
    dataset_eval, batch_size=batch_size, train=False, sort=False)

dl_test = torchtext.legacy.data.Iterator(
    dataset_test, batch_size=batch_size, train=False, sort=False)

# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": dl_train, "val": dl_eval}


In [24]:
# DataLoaderの動作確認 

batch = next(iter(dl_test))
print(batch)
print(batch.Text[0].shape)
print(batch.Label.shape)


[torchtext.legacy.data.batch.Batch of size 16]
	[.Text]:('[torch.LongTensor of size 16x512]', '[torch.LongTensor of size 16]')
	[.Label]:[torch.LongTensor of size 16]
torch.Size([16, 512])
torch.Size([16])


## 準備3：BERTのクラス分類用のモデルを用意する

Huggingfaceさんのをそのまま使うのではなく、BERTのbaseだけ使い、残りは自分で実装する

In [25]:
# from transformers.modeling_bert import BertModel
from transformers import BertModel #BertForMaskedLM

# BERTの日本語学習済みパラメータのモデルです
model = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
print(model)


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [26]:
from torch import nn


class BertForLivedoor(nn.Module):
    '''BERTモデルにLivedoorニュースの9クラスを判定する部分をつなげたモデル'''

    def __init__(self):
        super(BertForLivedoor, self).__init__()

        # BERTモジュール
        self.bert = model  # 日本語学習済みのBERTモデル

        # headにポジネガ予測を追加
        # 入力はBERTの出力特徴量の次元768、出力は9クラス
        self.cls = nn.Linear(in_features=768, out_features=9)

        # 重み初期化処理
        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)

    def forward(self, input_ids):
        '''
        input_ids： [batch_size, sequence_length]の文章の単語IDの羅列
        '''

        # BERTの基本モデル部分の順伝搬
        # 順伝搬させる
        result = self.bert(input_ids)  # reult は、sequence_output, pooled_output

        # sequence_outputの先頭の単語ベクトルを抜き出す
        vec_0 = result[0]  # 最初の0がsequence_outputを示す
        vec_0 = vec_0[:, 0, :]  # 全バッチ。先頭0番目の単語の全768要素
        vec_0 = vec_0.view(-1, 768)  # sizeを[batch_size, hidden_size]に変換
        output = self.cls(vec_0)  # 全結合層

        return output


In [27]:
# モデル構築
net = BertForLivedoor()

# 訓練モードに設定
net.train()

print('ネットワーク設定完了')


ネットワーク設定完了


## 準備4：BERTのファインチューニングの設定

In [28]:
# 勾配計算を最後のBertLayerモジュールと追加した分類アダプターのみ実行

# 1. まず全部を、勾配計算Falseにしてしまう
for param in net.parameters():
    param.requires_grad = False

# 2. BertLayerモジュールの最後を勾配計算ありに変更
for param in net.bert.encoder.layer[-1].parameters():
    param.requires_grad = True

# 3. 識別器を勾配計算ありに変更
for param in net.cls.parameters():
    param.requires_grad = True


In [29]:
# 最適化手法の設定
import torch.optim as optim


# BERTの元の部分はファインチューニング
optimizer = optim.Adam([
    {'params': net.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': net.cls.parameters(), 'lr': 1e-4}
])

# 損失関数の設定
criterion = nn.CrossEntropyLoss()
# nn.LogSoftmax()を計算してからnn.NLLLoss(negative log likelihood loss)を計算

## 5. 訓練を実施

In [30]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # ミニバッチのサイズ
    batch_size = dataloaders_dict["train"].batch_size

    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            iteration = 1

            # データローダーからミニバッチを取り出すループ
            for batch in (dataloaders_dict[phase]):
                # batchはTextとLableの辞書型変数

                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device)  # 文章
                labels = batch.Label.to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # BERTに入力
                    outputs = net(inputs)

                    loss = criterion(outputs, labels)  # 損失を計算

                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                        if (iteration % 10 == 0):  # 10iterに1度、lossを表示
                            acc = (torch.sum(preds == labels.data)
                                   ).double()/batch_size
                            print('イテレーション {} || Loss: {:.4f} || 10iter. || 本イテレーションの正解率：{}'.format(
                                iteration, loss.item(),  acc))

                    iteration += 1

                    # 損失と正解数の合計を更新
                    epoch_loss += loss.item() * batch_size
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))

    return net


In [31]:
# 学習・検証を実行する。1epochに2分ほどかかります
num_epochs = 4
net_trained = train_model(net, dataloaders_dict,
                          criterion, optimizer, num_epochs=num_epochs)


使用デバイス： cuda:0
-----start-------
イテレーション 10 || Loss: 2.0011 || 10iter. || 本イテレーションの正解率：0.375
イテレーション 20 || Loss: 2.0438 || 10iter. || 本イテレーションの正解率：0.3125
イテレーション 30 || Loss: 1.4062 || 10iter. || 本イテレーションの正解率：0.6875
イテレーション 40 || Loss: 1.2296 || 10iter. || 本イテレーションの正解率：0.5625
イテレーション 50 || Loss: 1.5334 || 10iter. || 本イテレーションの正解率：0.5625
イテレーション 60 || Loss: 1.0354 || 10iter. || 本イテレーションの正解率：0.8125
イテレーション 70 || Loss: 1.0237 || 10iter. || 本イテレーションの正解率：0.625
イテレーション 80 || Loss: 0.9275 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 90 || Loss: 0.3846 || 10iter. || 本イテレーションの正解率：0.875
イテレーション 100 || Loss: 0.5344 || 10iter. || 本イテレーションの正解率：0.875
イテレーション 110 || Loss: 0.4774 || 10iter. || 本イテレーションの正解率：0.875
イテレーション 120 || Loss: 0.5877 || 10iter. || 本イテレーションの正解率：0.75
イテレーション 130 || Loss: 0.3691 || 10iter. || 本イテレーションの正解率：0.875
イテレーション 140 || Loss: 0.7450 || 10iter. || 本イテレーションの正解率：0.6875
イテレーション 150 || Loss: 0.6219 || 10iter. || 本イテレーションの正解率：0.6875
イテレーション 160 || Loss: 0.5144 || 10iter. || 本イテレーションの正解率：0

## テストデータでの性能を確認

In [32]:
from tqdm import tqdm

# テストデータでの正解率を求める
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net_trained.eval()   # モデルを検証モードに
net_trained.to(device)  # GPUが使えるならGPUへ送る

# epochの正解数を記録する変数
epoch_corrects = 0

for batch in tqdm(dl_test):  # testデータのDataLoader
    # batchはTextとLableの辞書オブジェクト
    # GPUが使えるならGPUにデータを送る
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    inputs = batch.Text[0].to(device)  # 文章
    labels = batch.Label.to(device)  # ラベル

    # 順伝搬（forward）計算
    with torch.set_grad_enabled(False):

        # BertForLivedoorに入力
        outputs = net_trained(inputs)

        loss = criterion(outputs, labels)  # 損失を計算
        _, preds = torch.max(outputs, 1)  # ラベルを予測
        epoch_corrects += torch.sum(preds == labels.data)  # 正解数の合計を更新

# 正解率
epoch_acc = epoch_corrects.double() / len(dl_test.dataset)

print('テストデータ{}個での正解率：{:.4f}'.format(len(dl_test.dataset), epoch_acc))


100%|██████████| 93/93 [00:27<00:00,  3.34it/s]

テストデータ1475個での正解率：0.9146


https://yoheikikuta.github.io/bert-japanese/

https://github.com/yoheikikuta/bert-japanese

の「BERT with SentencePiece for Japanese text.」

では、入力テキストにタイトルを含めていますが、今回はタイトルは除いています。

同様にタイトルを抜いている、[BERTを用いた日本語文書分類タスクの学習・ハイパーパラメータチューニングの実践例](https://medium.com/karakuri/bert%E3%82%92%E7%94%A8%E3%81%84%E3%81%9F%E6%97%A5%E6%9C%AC%E8%AA%9E%E6%96%87%E6%9B%B8%E5%88%86%E9%A1%9E%E3%82%BF%E3%82%B9%E3%82%AF%E3%81%AE%E5%AD%A6%E7%BF%92-%E3%83%8F%E3%82%A4%E3%83%91%E3%83%BC%E3%83%91%E3%83%A9%E3%83%A1%E3%83%BC%E3%82%BF%E3%83%81%E3%83%A5%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0%E3%81%AE%E5%AE%9F%E8%B7%B5%E4%BE%8B-2fa5e4299b16)でも、正解率が92%ちょっととなっており、ほぼ同じ正解率が得られました。

以上。